In [1]:
import os, sys
from google.colab import drive
import shutil

drive.flush_and_unmount()
if os.path.isdir('/content/mnt')==False:
  drive.mount('/content/mnt')
  nb_path = '/content/notebooks'
  os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
  sys.path.insert(0, nb_path)  # or append(nb_path)

# Install only once. Tomorrow, you can skip this.
# !pip install --target=$nb_path spacy

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/mnt


In [2]:
import requests 
from bs4 import BeautifulSoup as bs
from fake_useragent import UserAgent
from googlesearch import search as gs
import re
import pandas

In [ ]:
# !pip install --target=$nb_path fake_useragent

In [3]:
ua = UserAgent()
headers = {
    'User-agent' : ua.random
}
year = 2000
url = f'http://billboardtop100of.com/{year}-2/'
r   = requests.get(url,headers = headers)

In [4]:
k = 0
id = 4748
songs_dict = []

for year in range(2000,2021):

  fail = False;

  print('--------------------------------------------')
  print(f'Scraping year {year}')
  print('--------------------------------------------')
  url = f'http://billboardtop100of.com/{year}-2/'

  r   = requests.get(url,headers = headers)
  soup = bs(r.text,'lxml')
  try : # page format with tables tr
    print('Method 1')
    print('--------------------------------------------')
    table = soup.find('table',{'class': ' alignleft'})
    total_song  = table.find_all('tr')

    # from year 2000

    for i in total_song:
      songs_dict.append({
      'Number' : k + 1,
      'Artist' : i.find_all('td')[1].text,
      'Song'   : i.find_all('td')[2].text,
      'Year'   : soup.title.text.split(' ')[0]
                
      })
      k+=1
      print(songs_dict[k-1]['Number'],songs_dict[k-1]['Artist'],songs_dict[k-1]['Song'])
  except:
     #page format changed
    print('Method 2 ')
    print('--------------------------------------------')
    try:

      songs = soup.find('small')
      pattern = '((\d)*. )'
      for i in songs.text.split('\n'):
        full_line = i.replace('\xa0','')
        p = re.match(pattern,full_line)
        line = full_line.replace(p[0],'')
        line = line.split('–')
        try:
          song =  line[1]
        except:
          song = 'NaN'
        songs_dict.append({
          'Number' : k + 1,
          'Artist' : line[0],
          'Song'   : song,
          'Year'   : soup.title.text.split(' ')[0]
                
          })
        k+=1
        print(songs_dict[k-1]['Number'],songs_dict[k-1]['Artist'],songs_dict[k-1]['Song'])


    except:
        try:
          #3rd try function.
          print('Method 3 ')
          print('--------------------------------------------')
          songs  = soup.find('table',{'class':'sortable alternate songtable'})
          tables = songs.find_all('tr')
          for i in tables:
            songs_dict.append({
            'Number' : k + 1,
            'Artist' : i.find_all('td')[1].text.replace('\n',''),
            'Song'   : i.find_all('td')[2].text.replace('\nLYRICS','').replace('\n',''),
            'Year'   : soup.title.text.split(' ')[0]
                
            
            })
            k+=1
            print(songs_dict[k-1]['Number'],songs_dict[k-1]['Artist'],songs_dict[k-1]['Song'])

        except:
          
          try:
            #4rd try function.
            print('Method 4 ')
            print('--------------------------------------------')
            songs  = soup.find_all('figure',{'class': 'wp-block-image'})
            for i in songs:
              song = i.find_next('p')
              artist = song.find_next('p')
              songs_dict.append({
              'Number' : k + 1,
              'Artist' : artist.text.replace('\n',''),
              'Song'   : song.text.replace('\n',''),
              'Year'   : soup.title.text.split(' ')[0]
                
              
              })
              k+=1
              print(songs_dict[k-1]['Number'],songs_dict[k-1]['Artist'],songs_dict[k-1]['Song'])
          finally:
            try:

              # 5th try function.
              print('Method 5 ')
              print('--------------------------------------------')
              songs  = soup.find_all('div',{'class': 'wp-block-media-text__content'})

              for i in songs:
                try :

                  song = i.find_all('strong')[1]
                  artist = i.find('a').text
                except:
                  song = i.find('p')
                  artist = song.find_next('p').text

                songs_dict.append({
                'Number' : k + 1,
                'Artist' : artist,
                'Song'   : song.text,
                'Year'   : soup.title.text.split(' ')[0]
                
                })
                k+=1
                print(songs_dict[k-1]['Number'],songs_dict[k-1]['Artist'],songs_dict[k-1]['Song'])
            finally:

                fail = True
                print('UGH THAT\'S LIKE 5 METHOD\'S')

  if fail ==True :
    print('Final Approach ')
    print('--------------------------------------------')
    try:
      songs  = soup.find_all('article',{'class': 'ye-chart-item'})

      for i in songs:
      
        song = i.find(class_ = 'ye-chart-item__title')
        artist = i.find(class_ = 'ye-chart-item__artist')


        songs_dict.append({
        'Number' : k + 1,
        'Artist' : artist.text,
        'Song'   : song.text,
        'Year'   : soup.title.text.split(' ')[0]
        
        })
        k+=1
        print(songs_dict[k-1]['Number'],songs_dict[k-1]['Artist'],songs_dict[k-1]['Song'])
    except:
      print('This is so long')

print('-----------------------------------------------------------------')
print(' End of Program.')
print(f' Amount of data scraped : {len(songs_dict)}')

--------------------------------------------
Scraping year 2000
--------------------------------------------
Method 1
--------------------------------------------
1 Faith Hill Breathe
2 Santana feat. Rob Thomas Smooth
3 Santana feat. The Product GandB Maria Maria
4 Joe I Wanna Know
5 Vertical Horizon Everything You Want
6 Destiny’s Child Say My Name
7 Savage Garden I Knew I Loved You
8 Lonestar Amazed
9 matchbox twenty Bent
10 Toni Braxton He Wasn’t Man Enough
11 Creed Higher
12 Aaliyah Try Again
13 Destiny’s Child Jumpin Jumpin’
14 Sisqo Thong Song
15 3 Doors Down Kryptonite
16 Pink There You Go
17 Madonna Music
18 Janet Doesn’t Really Matter
19 Christina Aguilera What A Girl Wants
20 Brian McKnight Back At One
21 ‘N Sync Bye Bye Bye
22 Marc Anthony You Sang To Me
23 Marc Anthony I Need To Know
24 Montell Jordan Get It On Tonite
25 Sisqo Incomplete
26 Macy Gray I Try
27 ‘N Sync It’s Gonna Be Me
28 Celine Dion That’s The Way It Is
29 Nelly (Hot S**T) Country Grammar
30 Blaque Bring It 

In [5]:
df = pandas.DataFrame(songs_dict)
df.to_csv(os.path.join('/content/mnt/MyDrive/Colab Notebooks','top100BillboardSongs.csv'))

In [ ]:
all_links   = []
ytlinks = []
count = 0;
for i in songs_dict:
  # i = 10
  Song = i['Song']
  Artist = i['Artist']

  Search = f'{Artist} - {Song} lyrics'
  print(f'{count}. Searched : {Search}')

  LINKS = list(gs(Search, tld='com',  num=10, start=0,stop=15,pause=2.0))
  res   = {
      'Results': LINKS,
      'Artist' : Artist,
      'Song'   : Song

  }
  all_links.append(res)
  count+=1

results = pandas.DataFrame(all_links)
results.to_csv(os.path.join('/content/mnt/MyDrive/Colab Notebooks','MusicLyricsLinks.csv'))

In [471]:
results = pandas.DataFrame(all_links)
results.to_csv(os.path.join('/content/mnt/MyDrive/Colab Notebooks','MusicLyricsLinks.csv'))

In [8]:
results.head()

,Results,Artist,Song
0,"[https://genius.com/Faith-hill-breathe-lyrics,...",Faith Hill,Breathe
1,"[https://www.youtube.com/watch?v=guG1JbozB2E, ...",Santana feat. Rob Thomas,Smooth
2,[https://genius.com/The-product-g-b-maria-mari...,Santana feat. The Product GandB,Maria Maria
3,"[https://genius.com/Joe-i-wanna-know-lyrics, h...",Joe,I Wanna Know
4,"[https://www.youtube.com/watch?v=rZoD8JEFjAE, ...",Vertical Horizon,Everything You Want


In [13]:
  for x in results : 
    print('-----------------------------------------')
    
]
    # for k in x['Results'] :
      # pattern_01 = r'\b(.azlyrics.)\b'
      # pattern_02 = r'\b(.youtube.)\b'
      # rmatch  = re.search(pattern_01,k)
      # youtube = re.search(pattern_02,k)
      # if rmatch :
      #   # links.append(k)
      #   print(k)
      # if youtube and count<1:
      #   count = 2
      #   #ytlinks.append(k)
      #   print(k)

SyntaxError: ignored

In [68]:

pattern_01 = r'\b(.azlyrics.)\b'
pattern_02 = r'\b(.youtube.)\b'
l = 1
m = 1
lyrics_links = []
ytlinks      = []
missing      = []
# youtube      = False
# found_ly     = False
# found_yt     = False
for i in range(0,len(results)):

  found_ly     = False
  found_yt     = False
  count = 0
  #print('--------------------------------------')
  for j in range(0,len(results[f'Results'][i])):
    k = results['Results'][i][j]
    rmatch  = re.search(pattern_01,k)
    youtube = re.search(pattern_02,k)
    if rmatch :
      found_ly = True
      lyrics_links.append(k)
      #print(l,'. ',k)
      l+=1
    
    if youtube and count<1:
      found_yt = True
      count = 2
      ytlinks.append(k)
      #print(m,'. ',k)
      m+=1
  if found_ly == False or found_yt == False:



    if found_ly == False:
      MISSING  = 'Lyrics Link'
    elif found_yt == False:
      MISSING  = 'Youtube Link'
    else:
      MISSING  = 'Both'
    MISS = {
        'Songs ' : results['Song'][i],
        'Artist' : results['Artist'][i],
        'Missing': MISSING

    }
    missing.append(MISS)

In [70]:
MISSING_DATA = pandas.DataFrame(missing)

In [72]:
MISSING_DATA[:87]

,Songs,Artist,Missing
0,Faded,SoulDecision feat. Thrust,Lyrics Link
1,Yes!,Chad Brock,Lyrics Link
2,I Hope You Dance,Lee Ann Womack and Sons Of The Desert,Lyrics Link
3,Better Off Alone,Alice Deejay,Lyrics Link
4,Dance With Me,Debelah Morgan,Lyrics Link
...,...,...,...
82,66 Popstar,DJ Khaled Featuring Drake,Lyrics Link
83,75 Suicidal,YNW Melly & Juice WRLD,Lyrics Link
84,81 Mood Swings,Pop Smoke Featuring Lil Tjay,Lyrics Link
85,84 Supalonely,BENEE Featuring Gus Dapperton,Lyrics Link


In [431]:
for x in links : 
  print('-----------------------------------------')
  for k in x :
    pattern = r'\b(azlyrics)\b'
    rmatch = re.search(pattern,k)
    if rmatch :

      print('Results : ', k)


-----------------------------------------
Results :  https://www.azlyrics.com/lyrics/faithhill/breathe.html
-----------------------------------------
Results :  https://www.azlyrics.com/lyrics/santana/smooth.html
-----------------------------------------
Results :  https://www.azlyrics.com/lyrics/santana/mariamaria.html


In [77]:
#export links : 
pandas.DataFrame(MISSING_DATA).to_csv('Missing_Links.csv',index = False)

In [ ]:
rmatch = re.match('azlyrics',k)

In [390]:

headers = {
    'authority': 'https://www.azlyrics.com',
    'dnt': '1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'none',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
}
link = 'https://www.azlyrics.com/lyrics/santana/mariamaria.html'

site = requests.get(link , headers = headers)
				


In [444]:
list(gs('LOVE', tld='com',  num=10, start=0,stop=7))

['https://www.merriam-webster.com/dictionary/love',
 'https://www.merriam-webster.com/dictionary/be%20in%20love',
 'https://www.merriam-webster.com/dictionary/love%27s%20test',
 'https://www.merriam-webster.com/dictionary/love%20affair',
 'https://www.merriam-webster.com/dictionary/fall%20in%20love',
 'https://www.dictionary.com/browse/love',
 'https://www.yahoo.com/lifestyle/love-list-star-hilary-farr-113000170.html']